# Bitcoin Project

In [ ]:
# pip install psycopg[binary]

### Import necessary libraries

In [ ]:
import requests
# import os
# from dotenv import load_dotenv
import psycopg
import json
from datetime import datetime
import pandas as pd
import streamlit as st

### Get API data 
Request a daily historical time series for a digital currency

In [ ]:
# Load environment variables from .env file
# load_dotenv()
api_key = st.secrets("BITCOIN_API")

In [ ]:
# Skonstruuj URL z kluczem API
url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey={api_key}"
response = requests.get(url)
print(response.status_code)
data = response.json()
print(data)

Convert a Python dict into a formatted JSON string for readability

In [ ]:
print(json.dumps(data, indent=2))

## Store data in an SQL database on Supabase

### Create a table function

In [ ]:
# load_dotenv()
dbconn = st.secrets("DB_CONN")

In [ ]:
# try:
#     conn = psycopg.connect(dbconn)
#     print("✅ Connection successful!")
#     conn.close()
# except Exception as e:
#     print("❌ Connection failed:")
#     print(e)

In [ ]:
def create_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    cur.execute(
        '''
            CREATE TABLE IF NOT EXISTS api_data(
                date TIMESTAMP PRIMARY KEY,
                open FLOAT,
                high FLOAT,
                low FLOAT,
                close FLOAT,
                volume FLOAT
            );
        '''
    )
    conn.commit()
    cur.close()
    conn.close()

create_table()

### Insert data into table

In [ ]:
conn = psycopg.connect(dbconn)
cur = conn.cursor()

time_series = data["Time Series (Digital Currency Daily)"]

for date_str, values in time_series.items():
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    open_price = float(values["1. open"])
    high_price = float(values["2. high"])
    low_price = float(values["3. low"])
    close_price = float(values["4. close"])
    volume = float(values["5. volume"])
    

    cur.execute(
    '''
        INSERT INTO api_data(date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (date) DO NOTHING;;
    ''', 
    (date_obj, open_price, high_price, low_price, close_price, volume)
    )

conn.commit()
cur.close()
conn.close()


#### Drop table if already exists

In [ ]:

# conn = psycopg.connect(dbconn)
# cur = conn.cursor()

# cur.execute("DROP TABLE IF EXISTS api_data;")

# conn.commit()
# cur.close()
# conn.close()

### Display latest data in a df to check it out

In [ ]:
# Connect to Supabase PostgreSQL
conn = psycopg.connect(dbconn)  # Make sure dbconn is defined with your Supabase URL and credentials
cur = conn.cursor()

# Run query
cur.execute("SELECT * FROM api_data ORDER BY date DESC LIMIT 10;")
rows = cur.fetchall()

# Close the connection
cur.close()
conn.close()


In [ ]:

# Display using pandas
df = pd.DataFrame(rows, columns=["date", "open", "high", "low", "close", "volume"])
print(df)